<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/32a_langchain_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ต่อไปเราจะมาเรียนรู้ LangChain

# 0. Magic Commands
ให้ใส่ไว้บนสุดทุก Notebook เป็นการสั่งให้ Notebook ก่อนรัน ให้รีโหลด Library ภายนอกที่เรา import ไว้ใหม่โดยอัตโนมัติ

และให้พล็อตกราฟ matplotlib ใน Output ของ cell แบบ code ได้เลย

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

สั่งให้ Colab แสดงผลข้อความแบบ Wrap Text

In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# 1. Import Library

ติดตั้ง OpenAI Library ด้วยการ Uncomment Cell ด้านล่างแล้ว Run

In [6]:
# ! pip install openai

# 2. Authentication

ก่อนที่เราจะเรียกใช้งาน ChatGPT, GPT-4 ของ OpenAI ได้ เราต้องลงทะเบียน สมัครใช้งาน และใส่ข้อมูลวิธีชำระเงินให้เรียบร้อยก่อน 

## Sign up

กดปุ่ม Sign up สีเขียว ในลิงค์ด้านล่าง สมัครใช้งาน OpenAI 

https://platform.openai.com/overview

ใส่ข้อมูลบัตรเครดิต ในหน้า Billing ให้เรียบร้อย

https://platform.openai.com/account/billing/payment-methods






## API Key

สร้าง API Key จากลิงค์ด้านล่าง โดยกดปุ่ม Create new secret key ตั้งชื่อ แล้ว Copy secret key เตรียมไว้ใส่ในช่อง OPENAI_API_KEY ด้านล่าง

https://platform.openai.com/account/api-keys


## ตัวอย่าง API Key 

OPENAI_API_KEY: sk-abcDeFGHijKlMNopQrstuvWxyZ1234567890abcDefhiJklm

รัน Cell ด้านล่าง แล้วใส่ secret key ที่ได้มา

บางที Run แล้วค้าง ไม่ขึ้นช่องให้กรอก ให้กดหยุด แล้วลอง Run ใหม่

In [8]:
import os
from getpass import getpass

token = getpass('Enter OPENAI_API_KEY here: ')
# print ('token is', token)

os.environ["OPENAI_API_KEY"] = token

Enter OPENAI_API_KEY here: ··········


หลังจากทดสอบเสร็จแล้ว แนะนำให้กลับไปลบ Secret Key ในหน้า https://platform.openai.com/account/api-keys ทิ้งไปทุกครั้ง เพื่อความปลอดภัย โดยเราสามารถสร้าง Key ใหม่ได้เรื่อย ๆ ไม่จำกัด

#3. Call API ตรง ๆ 

[จาก ep ก่อน Prompt Engineering คืออะไร ](https://www.bualabs.com/archives/4483/what-is-prompt-engineering/) เราได้เรียกใช้ chatgpt ผ่าน API ของ OpenAI ได้โดยตรง

In [11]:
import openai

In [12]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [22]:
country = "Thailand"
language = "English"

prompt = f"""Give me 10 reasons why {country} is a good place to retire.
Output in {language}.
"""

In [23]:
print(prompt)

Give me 10 reasons why Thailand is a good place to retire
Output in English



In [24]:
response = get_completion(prompt)

In [25]:
print(response)

1. Affordable cost of living: Thailand offers a low cost of living compared to many Western countries, making it an attractive option for retirees on a budget.

2. Warm climate: With its tropical climate, Thailand offers warm weather year-round, which can be especially appealing to retirees looking to escape colder climates.

3. High-quality healthcare: Thailand has a well-developed healthcare system, with modern facilities and highly trained medical professionals.

4. Friendly locals: Thai people are known for their warm hospitality and friendly nature, making it easy for retirees to feel welcome and at home.

5. Beautiful scenery: From stunning beaches to lush jungles, Thailand offers a diverse range of natural beauty that can be enjoyed by retirees.

6. Rich culture: Thailand has a rich cultural heritage, with ancient temples, traditional festivals, and a vibrant arts scene that can provide retirees with endless opportunities for exploration and learning.

7. Delicious food: Thai cu

In [32]:
country = "Singapore"
language = "English with stutter in every sentences"

prompt = f"""Give me 10 reasons why {country} is a good place to retire.
Output in {language}.
"""

In [33]:
print(prompt)

Give me 10 reasons why Singapore is a good place to retire.
Output in English with stutter in every sentences.



In [34]:
response = get_completion(prompt)
print(response)

W-w-well, S-Singapore is a g-g-great place to retire because:

1. It has a s-s-safe and secure environment.
2. The healthcare system is w-w-world-class.
3. The public transportation is efficient and accessible.
4. The c-c-climate is warm and tropical.
5. There are plenty of parks and green spaces to enjoy.
6. The food scene is diverse and delicious.
7. There are many cultural and recreational activities to participate in.
8. The economy is stable and prosperous.
9. The government provides support for seniors through various programs.
10. It's a m-m-multicultural society with a welcoming and friendly community.


#4. เรียกใช้ ChatGPT ผ่าน LangChain

เราสามารถใช้ F-String ในการสร้าง Template เพื่อสร้าง Prompt ส่งไปยัง ChatGPT ได้ไม่มีปัญหา แต่ถ้าเป็น Prompt ที่ซับซ้อนขึ้น มีหลาย Step และมี Output เป็นโครงสร้าง ที่จะนำไปใช้ต่อเช่น JSON การใช้คลาสเฉพาะ เช่น ChatPromptTemplate ของ LangChain สะดวกกว่า

In [ ]:
# https://python.langchain.com/en/latest/modules/models/llms/integrations.html

In [3]:
# ! pip install langchain

In [ ]:
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0.9)

In [ ]:
text = "What would be a good company name for a company that makes colorful wigs?"
print(llm(text))



FantasyFollicles.
